In [16]:
from google.colab import files
uploaded = files.upload()  # Upload your CSV file here

Saving flood_prediction.csv to flood_prediction (2).csv


In [17]:
import pandas as pd
df = pd.read_csv("flood_prediction.csv")  # replace with your actual filename
df.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,DrainageSystems,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability
0,3,8,6,6,4,4,6,2,3,2,...,10,7,4,2,3,4,3,2,6,0.450
1,8,4,5,7,7,9,1,5,5,4,...,9,2,6,2,1,1,9,1,3,0.475
2,3,10,4,1,7,5,4,7,4,9,...,7,4,4,8,6,1,8,3,6,0.515
3,4,4,2,7,3,4,1,4,6,4,...,4,2,6,6,8,8,6,6,10,0.520
4,3,7,5,2,5,8,5,2,7,5,...,7,6,5,3,3,4,4,3,4,0.475


In [18]:
!pip install pandas numpy scikit-learn streamlit pyngrok joblib


In [19]:
# Keep both columns: original probability + binary occurrence
df['Flood'] = (df['FloodProbability'] >= 0.5).astype(int)
df.head()

,MonsoonIntensity,TopographyDrainage,RiverManagement,Deforestation,Urbanization,ClimateChange,DamsQuality,Siltation,AgriculturalPractices,Encroachments,...,CoastalVulnerability,Landslides,Watersheds,DeterioratingInfrastructure,PopulationScore,WetlandLoss,InadequatePlanning,PoliticalFactors,FloodProbability,Flood
0,3,8,6,6,4,4,6,2,3,2,...,7,4,2,3,4,3,2,6,0.450,0
1,8,4,5,7,7,9,1,5,5,4,...,2,6,2,1,1,9,1,3,0.475,0
2,3,10,4,1,7,5,4,7,4,9,...,4,4,8,6,1,8,3,6,0.515,1
3,4,4,2,7,3,4,1,4,6,4,...,2,6,6,8,8,6,6,10,0.520,1
4,3,7,5,2,5,8,5,2,7,5,...,6,5,3,3,4,4,3,4,0.475,0


In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features: drop binary occurrence column
X = df.drop('Flood', axis=1)
y = df['Flood']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 1.0


In [22]:
import joblib, pickle

joblib.dump(model, 'model.pkl')
joblib.dump(scaler, 'scaler.pkl')

columns = X.columns.tolist()
with open('columns.pkl', 'wb') as f:
    pickle.dump(columns, f)


In [23]:
%%writefile app.py
import streamlit as st
import numpy as np
import joblib
import pickle

model = joblib.load('model.pkl')
scaler = joblib.load('scaler.pkl')
columns = pickle.load(open('columns.pkl','rb'))

st.title("Flood Prediction App")
st.write("Use sliders to enter environmental parameters:")

input_data = []
for col in columns:
    val = st.slider(f"{col}", 0, 10, 5)  # min=0, max=10, default=5
    input_data.append(val)

if st.button("Predict"):
    input_array = np.array(input_data).reshape(1, -1)
    input_scaled = scaler.transform(input_array)

    occurrence = model.predict(input_scaled)[0]
    predicted_probability = model.predict_proba(input_scaled)[0][1]

    st.subheader("Prediction Results")
    st.write("**Flood Occurrence:**", "Flood" if occurrence == 1 else "No Flood")
    st.write("**Predicted Flood Probability:**", f"{predicted_probability:.2f}")


Writing app.py


In [31]:
!pip install pyngrok


In [33]:
!ngrok authtoken 34bVuDhnyd8WhnLhYGh0Sb7u1SU_7x8trNwnQzfhAtqNTd473




Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [34]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print("Your Streamlit app URL:", public_url)


Your Streamlit app URL: NgrokTunnel: "https://christopher-manliest-jasmine.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py &




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.32.250.143:8501

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation